In [6]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import os
import json
import math
import numpy
import re
from tqdm import tqdm
import random
from scipy.io.wavfile import write

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [7]:
hps_mt = utils.get_hparams_from_file("./configs/genshin.json")

npcList = ['派蒙', '凯亚', '安柏', '丽莎', '琴', '香菱', '枫原万叶',
           '迪卢克', '温迪', '可莉', '早柚', '托马', '芭芭拉', '优菈',
           '云堇', '钟离', '魈', '凝光', '雷电将军', '北斗',
           '甘雨', '七七', '刻晴', '神里绫华', '戴因斯雷布', '雷泽',
           '神里绫人', '罗莎莉亚', '阿贝多', '八重神子', '宵宫',
           '荒泷一斗', '九条裟罗', '夜兰', '珊瑚宫心海', '五郎',
           '散兵', '女士', '达达利亚', '莫娜', '班尼特', '申鹤',
           '行秋', '烟绯', '久岐忍', '辛焱', '砂糖', '胡桃', '重云',
           '菲谢尔', '诺艾尔', '迪奥娜', '鹿野院平藏']

girl_npcList = ['派蒙', '丽莎', '琴', '香菱', '可莉', '早柚',  '芭芭拉', '优菈', '云堇', '凝光', '雷电将军', '北斗',
                '甘雨', '七七', '刻晴', '神里绫华', '罗莎莉亚', '八重神子', '宵宫', '九条裟罗', '夜兰', '珊瑚宫心海',
                '女士', '莫娜', '申鹤','行秋', '烟绯', '久岐忍', '辛焱', '砂糖', '胡桃', '重云','菲谢尔', '诺艾尔', '迪奥娜']

net_g_mt = SynthesizerTrn(
    len(symbols),
    hps_mt.data.filter_length // 2 + 1,
    hps_mt.train.segment_size // hps_mt.data.hop_length,
    n_speakers=hps_mt.data.n_speakers,
    **hps_mt.model).cuda()
_ = net_g_mt.eval()


_ = utils.load_checkpoint("./G_809000.pth", net_g_mt, None)

In [23]:
#读取每一行
lines=open("星临诸天.txt","r",encoding='utf8').readlines()
#验证（for循环遍历，不能修改列表）
# for line in tqdm(lines):
#     get_text(line.replace("\n", "").replace('\u3000',''),hps_mt)
chapters=[]
#分章
i=-1
for line in lines:
    if re.search("^第[0-9]+.*$",line):
        chapters.append([])
        i=i+1
        if i==0:
            chapters[0].append(line.replace("\n", "").replace('\u3000','')) 
        else:
            chapters[i].append(line.replace("\n", "").replace('\u3000',''))         
#     else:
#         chapters[i].append(line.replace("\n", "").replace('\u3000',''))

for chapter in tqdm(chapters,ncols=80):
    audio = numpy.array([])
    speaker=girl_npcList[random.randrange(0,len(girl_npcList))]
    for line in chapter:
        sentence = get_text(line.replace("\n", "").replace('\u3000',''), hps_mt)
        with torch.no_grad():
            inputs = sentence.cuda().unsqueeze(0)
            inputs_length = torch.LongTensor([sentence.size(0)]).cuda()
            sid = torch.LongTensor([npcList.index(speaker)]).cuda()
            audio =numpy.concatenate((audio,net_g_mt.infer(inputs, inputs_length, sid=sid, noise_scale=.4, noise_scale_w=.8, length_scale=1.1)[0][0,0].data.cpu().float().numpy()))
        outputfilename=speaker+"_"+chapter[0]+".wav"
        write(outputfilename, hps_mt.data.sampling_rate, audio)



100%|███████████████████████████████████████████| 38/38 [00:13<00:00,  2.83it/s]


In [ ]:
#循环部分人物合成
mynpcList=["派蒙"]
audio_mt = numpy.array([])
for speaker in tqdm(mynpcList):
    t_mt='I am happy'
    stn_tst_mt = get_text(t_mt.replace("\n", ""), hps_mt)
    with torch.no_grad():
        x_tst_mt = stn_tst_mt.cuda().unsqueeze(0)
        x_tst_mt_lengths = torch.LongTensor([stn_tst_mt.size(0)]).cuda()
        sid_mt = torch.LongTensor([npcList.index(speaker)]).cuda()
        audio_mt =numpy.concatenate((audio_mt,net_g_mt.infer(x_tst_mt, x_tst_mt_lengths, sid=sid_mt, noise_scale=.4, noise_scale_w=.8, length_scale=1.2)[0][0,0].data.cpu().float().numpy()))
ipd.display(speaker,ipd.Audio(audio_mt, rate=hps_mt.data.sampling_rate, normalize=False))